In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [7]:
class XODataset(Dataset):
    def __init__(self, file_path):
        self.data = []
        self.labels = []

        with open(file_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip().split()
            if len(line) == 101:
                image = np.array(line[:-1], dtype=np.float32).reshape(10, 10)
                label = int(line[-1])
                self.data.append(image)
                self.labels.append(0 if label == -1 else 1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        return image, label

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=2, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    for images, labels in train_loader:
        images = images.view(-1, 1, 10, 10).to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 1, 10, 10).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

In [13]:
if __name__ == "__main__":
    train_dataset = XODataset("C:\Dev\ANN\XOData10Train.txt")
    test_dataset = XODataset("C:\Dev\ANN\XOData10Test.txt")

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(10):
        train_model(model, train_loader, criterion, optimizer, device)
        print(f"Epoch {epoch+1}/10 completed")

    preds, labels = evaluate_model(model, test_loader, device)

    print("Confusion Matrix:")
    print(confusion_matrix(labels, preds))
    print("Classification Report:")
    print(classification_report(labels, preds))

Epoch 1/10 completed
Epoch 2/10 completed
Epoch 3/10 completed
Epoch 4/10 completed
Epoch 5/10 completed
Epoch 6/10 completed
Epoch 7/10 completed
Epoch 8/10 completed
Epoch 9/10 completed
Epoch 10/10 completed
Confusion Matrix:
[[22  3]
 [ 1 24]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.88      0.92        25
           1       0.89      0.96      0.92        25

    accuracy                           0.92        50
   macro avg       0.92      0.92      0.92        50
weighted avg       0.92      0.92      0.92        50

